### Test: Falcon 7B Model as Baseline

Unfortunately, even after a while of prompt engineering, this model seems to always output 'yes' and isn't any better than a random baseline.

In [1]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch

model_name = "tiiuae/falcon-7b-instruct"
CACHE_DIR = "/storage/courses/ds_workflow/car-web-scraping/model_cache"

# Load tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name, cache_dir=CACHE_DIR,)

# Load model with custom code (Flash Attention is not used by default on V100/older setups)
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype=torch.bfloat16 if torch.cuda.is_bf16_supported() else torch.float16,
    trust_remote_code=True,
    cache_dir=CACHE_DIR,
    device_map="auto"  # or just .cuda() if single-GPU
)

/storage/venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
A new version of the following files was downloaded from https://huggingface.co/tiiuae/falcon-7b-instruct:
- configuration_falcon.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


A new version of the following files was downloaded from https://huggingface.co/tiiuae/falcon-7b-instruct:
- modeling_falcon.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.
/storage/venv/lib/python3.11/site-packages/bitsandbytes/cextension.py:34: UserWarning: The installed version of bitsandbytes was compiled without GPU support. 8-bit optimizers, 8-bit multip


===================================BUG REPORT===================================
Welcome to bitsandbytes. For bug reports, please run

python -m bitsandbytes

 and submit this information together with your error trace to: https://github.com/TimDettmers/bitsandbytes/issues
bin /storage/venv/lib/python3.11/site-packages/bitsandbytes/libbitsandbytes_cpu.so
/storage/venv/lib/python3.11/site-packages/bitsandbytes/libbitsandbytes_cpu.so: undefined symbol: cadam32bit_grad_fp32
CUDA_SETUP: WARNING! libcudart.so not found in any environmental path. Searching in backup paths...
ERROR: /storage/venv/bin/python: undefined symbol: cudaRuntimeGetVersion
CUDA SETUP: libcudart.so path is None
CUDA SETUP: Is seems that your cuda installation is not in your path. See https://github.com/TimDettmers/bitsandbytes/issues/85 for more information.
CUDA SETUP: CUDA version lower than 11 are currently not supported for LLM.int8(). You will be only to use 8-bit optimizers and quantization routines!!
CUDA SETUP

Loading checkpoint shards: 100%|██████████| 2/2 [00:06<00:00,  3.29s/it]


In [4]:
# Long vehicle description
car_description = """
Category: Van / minibus, 5 door. Engine type: Diesel. Fuel type: Diesel. Emission class: EURO 6. 
CO2 emissions: 173 g/km (combined). Power output: 110 KW / 150 PS. First registration: 12.2023. 
KBA Key Manufacturer: 0603. KBA Key Type: CQJ. VIN: WV2ZZZST2RH******. Transmission: Automatic. 
Colour: white (Sonderlackierung Candy-weiss). Read mileage: 21,800 Kilometres. 
Owners: 1. Location: D-73. Vehicle release: 3 working days after payment.
"""

# Short search texts
queries = [
    # 1. Perfect match — fits the long description
    "I'm looking for a white diesel van with automatic transmission, registered recently, less than 30,000 km driven, and EURO 6 emission class.",
    
    # 2. Significant mismatch — different fuel, emission, transmission, and mileage
    "I'm searching for a white petrol van with manual transmission, EURO 5 emissions, and around 80,000 kilometers mileage.",
    
    # 3. Completely different vehicle category and features
    "I'm interested in a red convertible sports car with over 300 horsepower and leather seats.",

    ## New harder task:
    # 4. Close to match — but **fuel type is incorrect** (petrol instead of diesel)
    "I'm looking for a white petrol van with automatic transmission, registered recently, less than 30,000 km driven, and EURO 6 emission class.",
    
    ## New harder task:
    # 5. Mileage too strict — **less than 20,000 km** instead of 21,800 km
    "I'm looking for a white diesel van with automatic transmission, registered recently, less than 20,000 km driven, and EURO 6 emission class."
]

prompt = """
A car dealer compares if the description of a car is exactly what his customer asked for.
Does the question match the description: Please answer yes or no (yes/no): """

In [5]:
for query in queries:
    combined = f"Car description: {car_description} question: {query} --- {prompt}"
    inputs = tokenizer(combined, return_tensors="pt").to(model.device)

    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_new_tokens=5,
            do_sample=False
        )

    print(tokenizer.decode(outputs[0], skip_special_tokens=True))

Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.


Car description: 
Category: Van / minibus, 5 door. Engine type: Diesel. Fuel type: Diesel. Emission class: EURO 6. 
CO2 emissions: 173 g/km (combined). Power output: 110 KW / 150 PS. First registration: 12.2023. 
KBA Key Manufacturer: 0603. KBA Key Type: CQJ. VIN: WV2ZZZST2RH******. Transmission: Automatic. 
Colour: white (Sonderlackierung Candy-weiss). Read mileage: 21,800 Kilometres. 
Owners: 1. Location: D-73. Vehicle release: 3 working days after payment.
 question: I'm looking for a white diesel van with automatic transmission, registered recently, less than 30,000 km driven, and EURO 6 emission class. --- 
A car dealer compares if the description of a car is exactly what his customer asked for.
Does the question match the description: Please answer yes or no (yes/no): 
Yes
Car description: 
Category: Van / minibus, 5 door. Engine type: Diesel. Fuel type: Diesel. Emission class: EURO 6. 
CO2 emissions: 173 g/km (combined). Power output: 110 KW / 150 PS. First registration: 12.2023

Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.


Car description: 
Category: Van / minibus, 5 door. Engine type: Diesel. Fuel type: Diesel. Emission class: EURO 6. 
CO2 emissions: 173 g/km (combined). Power output: 110 KW / 150 PS. First registration: 12.2023. 
KBA Key Manufacturer: 0603. KBA Key Type: CQJ. VIN: WV2ZZZST2RH******. Transmission: Automatic. 
Colour: white (Sonderlackierung Candy-weiss). Read mileage: 21,800 Kilometres. 
Owners: 1. Location: D-73. Vehicle release: 3 working days after payment.
 question: I'm interested in a red convertible sports car with over 300 horsepower and leather seats. --- 
A car dealer compares if the description of a car is exactly what his customer asked for.
Does the question match the description: Please answer yes or no (yes/no): 
Yes
Car description: 
Category: Van / minibus, 5 door. Engine type: Diesel. Fuel type: Diesel. Emission class: EURO 6. 
CO2 emissions: 173 g/km (combined). Power output: 110 KW / 150 PS. First registration: 12.2023. 
KBA Key Manufacturer: 0603. KBA Key Type: CQJ.